## SQL COMMANDS

### Data Definition Language (DDL): Used to define database schema

DDL commands:</br>

CREATE – is used to create the database or its objects (like table, index, function, views, store procedure and triggers).</br>
DROP – is used to delete objects from the database.</br>
ALTER-is used to alter the structure of the database.</br>
TRUNCATE–is used to remove all records from a table, including all spaces allocated for the records are removed.</br>
**Now Lets See Some Examples**

In [67]:
import os
if os.path.exists("chinook.db"):
   os.remove('chinook.db')

import sqlite3
conn = sqlite3.connect('chinook.db')
cur=conn.cursor()

In [68]:
def execute(queryString):
  conn.execute(queryString)
  conn.commit()
  
  
def executemany(queryString,data):
  cur.executemany(queryString,data)
  conn.commit()

def showCursorData(cursor):
  print("\n")
  print("Result:")  
  for row in cursor:
    print(row)
  

def showData(tbl):
  query = 'SELECT * FROM {} ORDER BY 1'.format(tbl)
  cursor = conn.execute(query)
  for row in cursor:
    print(row)

def getColumnsInTable(tbl):
    query = 'SELECT * FROM {}'.format(tbl)
    cursor = conn.execute(query)
    print(list(map(lambda x: x[0], cursor.description)))

### CREATE TABLE 

In [69]:
conn.execute("""DROP table IF EXISTS regions""")
conn.execute("""DROP table IF EXISTS countries""")

query1 = '''
CREATE TABLE regions (
    region_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    region_name varchar(20) ,
    region_code varchar(20)
)
'''
execute(query1)
getColumnsInTable('regions')

['region_id', 'region_name', 'region_code']


In [70]:
# Let us create a Countries table that links regions table using a foreign key constraint
query2 = '''
CREATE TABLE countries (
    country_id text PRIMARY KEY  NOT NULL,
    region_id INTEGER NOT NULL,
    country_growthRate decimal(5,2) DEFAULT 0,  /* Decimal type conversion */
    FOREIGN KEY (region_id) REFERENCES regions (region_id) ON DELETE CASCADE ON UPDATE CASCADE
)
'''
execute(query2)
getColumnsInTable('countries')

['country_id', 'region_id', 'country_growthRate']


### ALTER TABLE

In [71]:
query= "ALTER TABLE countries ADD country_name text"
execute(query)
getColumnsInTable('countries')

['country_id', 'region_id', 'country_growthRate', 'country_name']


### DROP TABLE

In [72]:
query = "DROP TABLE countries"
execute(query)
getColumnsInTable('countries')

OperationalError: no such table: countries

### TRUNCATE TABLE : Removes the data retaining the structure of the table

In [73]:
# let us add some data before truncating
insQuery="INSERT INTO regions(region_id,region_name) VALUES (1,'Europe')"
execute(insQuery)
showData('regions')

(1, 'Europe', None)


* SQLite does not have an explicit TRUNCATE TABLE command like other databases
* Instead, it has added a TRUNCATE optimizer to the DELETE statement. 
* To truncate a table in SQLite, you just need to execute a DELETE statement without a WHERE clause. 
* The TRUNCATE optimizer handles the rest

In [74]:
query="DELETE FROM regions"
execute(query)
showData('regions')

### DML(Data Manipulation Language) : Deals with the manipulation of data present in database
DML COMMANDS : <BR/>
SELECT – is used to retrieve data from the a database.<BR/>
INSERT – is used to insert data into a table.<BR/>
UPDATE – is used to update existing data within a table.<BR/>
DELETE – is used to delete records from a database table.<BR/>

### INSERT

In [75]:
insQuery="INSERT INTO regions VALUES (1,'Europe','001')"
execute(insQuery)

# Inserting NULL values
insQuery2="INSERT INTO regions VALUES (2,'Argentina',NULL)"
execute(insQuery2)


# Insert values in specific columns
insQuery3="INSERT INTO regions(region_name,region_code) VALUES ('Asia','002')"
execute(insQuery3)
showData("regions")

(1, 'Europe', '001')
(2, 'Argentina', None)
(3, 'Asia', '002')


### SELECT

In [76]:
query="select * from regions"
cursor = conn.execute(query)
showCursorData(cursor)

# select using a condition

query="select * from regions where region_code>='002'"
cursor = conn.execute(query)
showCursorData(cursor)

  



Result:
(1, 'Europe', '001')
(2, 'Argentina', None)
(3, 'Asia', '002')


Result:
(3, 'Asia', '002')


### UPDATE

In [77]:
# Updating single column
updateQuery="UPDATE regions SET region_code='003' where region_id=2"
execute(updateQuery)
showData('regions')

(1, 'Europe', '001')
(2, 'Argentina', '003')
(3, 'Asia', '002')


In [78]:
# Updating multiple columns
updateQuery2="UPDATE regions SET region_code='004',region_name='America' where region_id=2"
execute(updateQuery2)
showData('regions')

# If we omit the WHERE clause from the update query then all of the rows will get updated.

(1, 'Europe', '001')
(2, 'America', '004')
(3, 'Asia', '002')


### DELETE  : *Used to delete the existing records from a table.*

In [79]:
delQuery="DELETE FROM regions where region_id>2"
execute(delQuery)
showData('regions')

# If we omit the WHERE clause from the delete query then all of the records will get deleted.


(1, 'Europe', '001')
(2, 'America', '004')


### JOINS :
Three major types of joins −

The CROSS JOIN<br/>
The INNER JOIN<br/>
The OUTER JOIN<br/>

__Before we proceed , lets create two tables and populate data to it. __

In [80]:
conn.execute("""DROP table IF EXISTS departments""")
conn.execute("""DROP table IF EXISTS employees""")

query= '''
CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    department_name text NOT NULL
)
'''
execute(query)

query = '''
CREATE TABLE employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    first_name text,
    last_name text NOT NULL,
    email text NOT NULL,
    phone_number text,
    hire_date text NOT NULL,
    emp_enroll_id INTEGER NOT NULL,
    salary double NOT NULL,
    manager_id INTEGER,
    dep_id INTEGER NOT NULL,
    FOREIGN KEY (dep_id) REFERENCES departments (department_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (manager_id) REFERENCES employees (employee_id) ON DELETE CASCADE ON UPDATE CASCADE
)
'''
execute(query)



# lets populate some data

dep_ins="INSERT INTO departments(department_id,department_name) VALUES (?,?)"
dep_data=[(1,'Administration'),(2,'Marketing'),(3,'Purchasing')]
executemany(dep_ins,dep_data)

emp_ins="INSERT INTO employees(employee_id,first_name,last_name,email,phone_number,hire_date,emp_enroll_id,salary,manager_id,dep_id) VALUES (?,?,?,?,?,?,?,?,?,?)"
emp_data=[(100,'Steven','King','steven.king@xyz.com','515.123.4567','1987-06-17',4,24000.00,0,1),
         (101,'Neena','Kochhar','neena.kochhar@xyz.com','515.123.4568','1989-09-21',5,17000.00,100,2),
         (102,'Lex','De Haan','lex.de haan@xyz.com','515.123.4569','1993-01-13',5,17000.00,100,3),
         (103,'Alexander','Hunold','alexander.hunold@xyz.com','590.423.4567','1990-01-03',9,9000.00,102,2),
         (104,'Bruce','Ernst','bruce.ernst@xyz.com','590.423.4568','1991-05-21',9,6000.00,103,1),
         (105,'David','Austin','david.austin@xyz.com','590.423.4569','1997-06-25',9,4800.00,103,4)]
executemany(emp_ins,emp_data)

In [81]:
showData('employees')

(100, 'Steven', 'King', 'steven.king@xyz.com', '515.123.4567', '1987-06-17', 4, 24000.0, 0, 1)
(101, 'Neena', 'Kochhar', 'neena.kochhar@xyz.com', '515.123.4568', '1989-09-21', 5, 17000.0, 100, 2)
(102, 'Lex', 'De Haan', 'lex.de haan@xyz.com', '515.123.4569', '1993-01-13', 5, 17000.0, 100, 3)
(103, 'Alexander', 'Hunold', 'alexander.hunold@xyz.com', '590.423.4567', '1990-01-03', 9, 9000.0, 102, 2)
(104, 'Bruce', 'Ernst', 'bruce.ernst@xyz.com', '590.423.4568', '1991-05-21', 9, 6000.0, 103, 1)
(105, 'David', 'Austin', 'david.austin@xyz.com', '590.423.4569', '1997-06-25', 9, 4800.0, 103, 4)


In [82]:
showData('departments')

(1, 'Administration')
(2, 'Marketing')
(3, 'Purchasing')


### CROSS JOIN 
Matches every row of the first table with every row of the second table. If the input tables have x and y row, respectively, the resulting table will have x*y row. Because CROSS JOINs have the potential to generate extremely large tables, care must be taken to only use them when appropriate.

In [83]:
query = '''SELECT employee_id,first_name, department_name FROM employees CROSS JOIN departments'''
cursor = conn.execute(query)
showCursorData(cursor)




Result:
(100, 'Steven', 'Administration')
(100, 'Steven', 'Marketing')
(100, 'Steven', 'Purchasing')
(101, 'Neena', 'Administration')
(101, 'Neena', 'Marketing')
(101, 'Neena', 'Purchasing')
(102, 'Lex', 'Administration')
(102, 'Lex', 'Marketing')
(102, 'Lex', 'Purchasing')
(103, 'Alexander', 'Administration')
(103, 'Alexander', 'Marketing')
(103, 'Alexander', 'Purchasing')
(104, 'Bruce', 'Administration')
(104, 'Bruce', 'Marketing')
(104, 'Bruce', 'Purchasing')
(105, 'David', 'Administration')
(105, 'David', 'Marketing')
(105, 'David', 'Purchasing')


### INNER JOIN
INNER JOIN creates a new result table by combining column values of two tables (table1 and table2) based upon the join-predicate. 
Following is the syntax of INNER JOIN −

**SELECT ... FROM table1 [INNER] JOIN table2 ON conditional_expression **...<br/>
To avoid redundancy and keep the phrasing shorter, INNER JOIN conditions can be declared with a USING expression. This expression specifies a list of one or more columns.

**SELECT ... FROM table1 JOIN table2 USING ( column1 ,... ) **...<br/>
A NATURAL JOIN is similar to a JOIN...USING, only it automatically tests for equality between the values of every column that exists in both tables −

**SELECT ... FROM table1 NATURAL JOIN table2.**...<br/>
Based on the above tables, you can write an INNER JOIN as follows −

In [84]:
query = '''SELECT employee_id,first_name, department_name FROM employees INNER JOIN departments ON employees.dep_id=departments.department_id'''
cursor = conn.execute(query)
showCursorData(cursor)




Result:
(100, 'Steven', 'Administration')
(101, 'Neena', 'Marketing')
(102, 'Lex', 'Purchasing')
(103, 'Alexander', 'Marketing')
(104, 'Bruce', 'Administration')


### OUTER JOINS
There are three types of OUTER JOINs: LEFT, RIGHT, and FULL


OUTER JOINs have a condition that is identical to INNER JOINs, expressed using an ON, USING, or NATURAL keyword. The initial results table is calculated the same way. Once the primary JOIN is calculated, an OUTER JOIN will take any unjoined rows from one or both tables, pad them out with NULLs, and append them to the resulting table.



In [85]:
# To show example, SQLite only supports the LEFT OUTER JOIN.
query = '''SELECT employee_id,first_name, department_name FROM employees LEFT JOIN departments ON employees.dep_id=departments.department_id'''
cursor = conn.execute(query)
showCursorData(cursor)



Result:
(100, 'Steven', 'Administration')
(101, 'Neena', 'Marketing')
(102, 'Lex', 'Purchasing')
(103, 'Alexander', 'Marketing')
(104, 'Bruce', 'Administration')
(105, 'David', None)


### SUBQUERY
A Subquery or Inner query or a Nested query is a query within another SQL query and embedded within the WHERE clause.

A subquery is used to return data that will be used in the main query as a condition to further restrict the data to be retrieved.

Subqueries can be used with the SELECT, INSERT, UPDATE, and DELETE statements along with the operators like =, <, >, >=, <=, IN, BETWEEN, etc.



In [86]:
# EXAMPLE
query = '''SELECT employee_id,first_name,email FROM employees WHERE employee_id IN (SELECT employee_id FROM employees WHERE salary>10000)'''
cursor = conn.execute(query)
showCursorData(cursor)



Result:
(100, 'Steven', 'steven.king@xyz.com')
(101, 'Neena', 'neena.kochhar@xyz.com')
(102, 'Lex', 'lex.de haan@xyz.com')


### UNION
UNION clause/operator is used to combine the results of two or more SELECT statements without returning any duplicate rows.

To use this UNION clause, each SELECT statement must have

The same number of columns selected

The same number of column expressions

The same data type and

Have them in the same order

But they need not have to be in the same length.

In [87]:

query = '''SELECT employee_id,first_name, department_name FROM employees INNER JOIN departments 
ON employees.dep_id=departments.department_id
UNION
SELECT employee_id,first_name, department_name FROM employees LEFT JOIN departments
ON employees.dep_id=departments.department_id
'''
cursor = conn.execute(query)
showCursorData(cursor)




Result:
(100, 'Steven', 'Administration')
(101, 'Neena', 'Marketing')
(102, 'Lex', 'Purchasing')
(103, 'Alexander', 'Marketing')
(104, 'Bruce', 'Administration')
(105, 'David', None)


### The UNION ALL Clause
The UNION ALL operator is used to combine the results of two SELECT statements including duplicate rows.

The same rules that apply to UNION apply to the UNION ALL operator as well.

In [88]:
query = '''SELECT employee_id,first_name, department_name FROM employees INNER JOIN departments 
ON employees.dep_id=departments.department_id
UNION ALL
SELECT employee_id,first_name, department_name FROM employees LEFT JOIN departments
ON employees.dep_id=departments.department_id
'''
cursor = conn.execute(query)
showCursorData(cursor)



Result:
(100, 'Steven', 'Administration')
(101, 'Neena', 'Marketing')
(102, 'Lex', 'Purchasing')
(103, 'Alexander', 'Marketing')
(104, 'Bruce', 'Administration')
(100, 'Steven', 'Administration')
(101, 'Neena', 'Marketing')
(102, 'Lex', 'Purchasing')
(103, 'Alexander', 'Marketing')
(104, 'Bruce', 'Administration')
(105, 'David', None)
